In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=9d2b308f6c2477ddfcd32e7aa4347822196156be4a53feb7a30f239d29aba9fc
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql.functions import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
c = pyspark.SparkConf().setAppName("test_app").setMaster("local")
sc = pyspark.SparkContext(conf = c)
spark = SparkSession(sc)
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

In [3]:
from pyspark.sql.types import *


In [4]:
data = spark.read.csv('/content/drive/MyDrive/Copy of accepted_2007_to_2018Q4.csv', header = True, inferSchema = True, escape = '"')

In [5]:
data.show(10,False)

+--------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+---------------------------------------+----------+--------------+----------+-------------------+--------+-----------+----------+-----------------------------------------------------------------+----+------------------+-----------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+------------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+--

In [6]:
data.count(), len(data.columns)

(2260701, 151)

In [7]:
data.columns

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 '

In [8]:
A25 = ['id','member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url','desc', 'purpose', 'title', 'zip_code', 'addr_state','dti']
B25 = ['delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d']
C25 = ['last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m','max_bal_bc','all_util', 'total_rev_hi_lim']
D25 = ['inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl']
E25 = ['num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort','total_bc_limit', 'total_il_high_credit_limit', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts']
F25 = ['sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'hardship_flag', 'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 'disbursement_method', 'debt_settlement_flag', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date', 'settlement_amount','settlement_percentage','settlement_term']

In [9]:
len(A25),len(B25),len(C25),len(D25),len(E25),len(F25)

(25, 25, 25, 25, 25, 26)

In [10]:
data.select(A25).show(5,False)

+--------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+---------------------------+----------+--------------+----------+-------------------+--------+-----------+----------+-----------------------------------------------------------------+----+------------------+------------------+--------+----------+-----+
|id      |member_id|loan_amnt|funded_amnt|funded_amnt_inv|term      |int_rate|installment|grade|sub_grade|emp_title                  |emp_length|home_ownership|annual_inc|verification_status|issue_d |loan_status|pymnt_plan|url                                                              |desc|purpose           |title             |zip_code|addr_state|dti  |
+--------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+---------------------------+----------+--------------+----------+-------------------+--------+-----------+----------+-----------------------------------------------------------

In [11]:
# Data Cleaning
# How to find the Missing Values
#print(data.filter(col('id').isNull()).count())
#print(data.filter(col('member_id').isNull()).count())

In [12]:
# Missing Values (A25)
#data.select([count(when (isnan(c) | col(c).isNull(), c)).alias(c) for c in A25]).show()

In [13]:
# Missing Values Percentage
#data.select([round (((count(when(isnan(c) | col(c).isNull() ,c))/data.count()) * 100 ),2).alias(c) for c in A25 ]).show()

In [14]:
# From A25 columns , columns_drop = ['member_id','desc'] having null > 75%

In [15]:
#data.select([count(when (isnan(c) | col(c).isNull(), c)).alias(c) for c in B25]).show()

In [16]:
# Missing Values Percentage for B25
#data.select([round(( (count(when(isnan(c) | col(c).isNull(),c )) /data.count()) * 100),2).alias(c) for c in B25]  ).show()

In [17]:
# Missing Values Percentage for C25
#data.select([round(( (count(when(isnan(c) | col(c).isNull(),c))/data.count() ) *100 ),2).alias(c) for c in C25]).show()

In [18]:
# Missing Values Percentage for D25
#data.select([round( ((count(when(isnan(c) | col(c).isNull(),c))/data.count())*100 ) ,2).alias(c) for c in D25]).show()

In [19]:
# Missing Values Percentage for F25
#data.select([round(((count(when(isnan(c) | col(c).isNull(),c))/data.count())*100 ) ,2).alias(c)for c in F25]).show()

In [20]:
# Missing Values Percentage for E25
#data.select([round( (( count(when(isnan(c) | col(c).isNull(),c ) )/data.count()) *100),2).alias(c) for c in E25]).show()

In [21]:
# columns to drop having null > 70%  from
# A25  ['member_id','desc']
# B25 ['mths_since_last_record']
# C25 ['mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint']
# D25 ['mths_since_recent_bc_dlq']
# E25 ['revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths',
#       'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts']
# F25 ['sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'hardship_type',
#       'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date',
#       'hardship_length', 'hardship_dpd', 'hardship_loan_status', 'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount',
#       'hardship_last_payment_amount', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date', 'settlement_amount', 'settlement_percentage', 'settlement_term' ]

In [22]:
columns_to_drop = ['member_id','desc','mths_since_last_record', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint',
                   'mths_since_recent_bc_dlq','revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line',
                   'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts',
                   'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog',
                   'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date',
                   'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status',
                   'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 'debt_settlement_flag_date',
                   'settlement_status', 'settlement_date', 'settlement_amount', 'settlement_percentage', 'settlement_term']
data1 = data.drop(*columns_to_drop)

In [23]:
len(columns_to_drop)

41

In [24]:
len(data.columns) - len(columns_to_drop)

110

In [25]:
len(data.columns), len(data1.columns)

(151, 110)

In [26]:
print(data1.select('term').distinct().show())

+----------+
|      term|
+----------+
| 36 months|
| 60 months|
|      NULL|
+----------+

None


In [27]:
data1 = data1.withColumn('term_in_months', regexp_replace(col('term'),'months',''))

In [28]:
#print(data1.select('term_in_months','term').distinct().show())

In [29]:
data1 = data1.drop('term')

In [30]:
# Column emp_length
# data1.select('emp_length').distinct().show(13, False)

In [31]:
data1.select('emp_length').distinct().count()

12

In [32]:
data1.select('id').filter(col('emp_length').like('%reactors%')).count()
# spark.read.csv(''data.csv', header = True, inferschema=True, Escape = '""') # This will eliminate the character with "" in csv data

0

In [33]:
data1.select('emp_length').distinct().show() # reactor is not apearing

+----------+
|emp_length|
+----------+
|   5 years|
|   9 years|
|    1 year|
|   2 years|
|   7 years|
|   8 years|
|   4 years|
|   6 years|
|   3 years|
| 10+ years|
|  < 1 year|
|      NULL|
+----------+



In [34]:
data1= data1.withColumn('emp_length_years', regexp_extract(col('emp_length'),'\\d+',0))  # regexp_extract(str, pattern, index)

In [35]:
data1.select('emp_length_years', 'emp_length').distinct().show(10)

+----------------+----------+
|emp_length_years|emp_length|
+----------------+----------+
|               1|  < 1 year|
|               6|   6 years|
|               1|    1 year|
|               8|   8 years|
|               7|   7 years|
|               4|   4 years|
|               5|   5 years|
|               9|   9 years|
|               3|   3 years|
|              10| 10+ years|
+----------------+----------+
only showing top 10 rows



In [36]:
data1 = data1.drop('emp_length')

In [37]:
data1 = data1.fillna(0, subset = ['emp_length_years'])  # subset can be used for multiple columns

In [38]:
data1.select('emp_length_years').distinct().show(100)

+----------------+
|emp_length_years|
+----------------+
|               7|
|               3|
|               8|
|               5|
|               6|
|               9|
|               1|
|              10|
|               4|
|               2|
|            NULL|
+----------------+



In [39]:
# data1 = data1.withColumn('emp_length_years', col('emp_length_years').cast(int))
# from pyspark.sql.types import *
# from pyspark.sql.functions import *

In [40]:
data1 = data1.withColumn('emp_length_years', data1['emp_length_years'].cast(IntegerType()))

In [41]:
data1 = data1.fillna(0, subset = ['emp_length_years'])
data1.select('emp_length_years').distinct().show(100) #  null value replaced with '0' in the distinct values

+----------------+
|emp_length_years|
+----------------+
|               1|
|               6|
|               3|
|               5|
|               9|
|               4|
|               8|
|               7|
|              10|
|               2|
|               0|
+----------------+



In [42]:
# emp_title
# data1.select('emp_title').distinct().show()

In [43]:
data1.filter(col('emp_title').isNull()).count()
# 167002

167002

In [44]:
(167002/data1.count())*100
# 7% null value, so impute the values 'NA'.

7.387177693998455

In [45]:
# data1 = data1.fillna('N/A', subset = ['emp_title'])
# missing value handeled

In [46]:
#data1.filter(col('emp_title').isNull()).count()

In [47]:
#data1.select('home_ownership').distinct().show(10)

In [48]:
#data1 = data1.fillna('N/A', subset = ['emp_title','home_ownership'])

In [49]:
# data1.select('home_ownership').distinct().show(40)
# data1.select('verification_status').distinct().show(40)

In [50]:
# data1.select('grade').distinct().show(10)
# data1.select('sub_grade').distinct().show(10)

In [51]:
 #data1.select('issue_d').distinct().show() # checking the distinct column for all the A13 Columns

In [52]:
# data1.filter(col('url').isNull()).count()
# 33
data1.select('url').distinct().count()

2260669

In [53]:
# Original list of column names
column_names = data1.columns

# Number of sublists you want
num_sublists = 13

# Calculate the size of each sublist
sublist_size = len(column_names) // num_sublists

# Divide the list into sublists
sublists = [column_names[i:i + sublist_size] for i in range(0, len(column_names), sublist_size)]

# Store sublists in new variables with meaningful names
for i, sublist in enumerate(sublists):
    new_variable_name = f'N_column'
    globals()[new_variable_name] = sublist

# Print the sublists
for i, sublist in enumerate(sublists):
    print(f'{new_variable_name}_{i + 1}: {sublist}')


N_column_1: ['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'grade', 'sub_grade']
N_column_2: ['emp_title', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url']
N_column_3: ['purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low']
N_column_4: ['fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc']
N_column_5: ['initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee']
N_column_6: ['recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low']
N_column_7: ['collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il']
N_column_8

In [54]:
len(data1.columns)

110

In [55]:
N_column_1 = ['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'grade', 'sub_grade']
N_column_2 = ['emp_title', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url']
N_column_3 = ['purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low']
N_column_4 = ['fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc']
N_column_5 = ['initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee']
N_column_6 = ['recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low']
N_column_7 = ['collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il']
N_column_8 = ['open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc']
N_column_9 = ['all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy']
N_column_10 = ['bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc']
N_column_11 = ['mths_since_recent_bc', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl']
N_column_12 = ['num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m']
N_column_13 = ['num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit']
N_column_14 = ['total_il_high_credit_limit', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag', 'term_in_months', 'emp_length_years']

In [98]:
# N_column_1 - Zero null values
# N_column_2 - Zero null values
# N_column_3 - Zero null values
# N_column_4 - Zero null values
# N_column_5 - Zero null values
# N_column_6 - Zero null values
# N_column_7 - Zero null values
# N_column_8 - Zero null values
# N_column_9 - Zero null values
# N_column_10 - Zero null values
# N_column_11 - Zero null values
# N_column_12 - Zero null values
# N_column_13 - Zero null values
# N_column_14 - Zero null values


data1.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c)  for c in N_column_14]).show()

+--------------------------+-------------+-------------------+--------------------+--------------+----------------+
|total_il_high_credit_limit|hardship_flag|disbursement_method|debt_settlement_flag|term_in_months|emp_length_years|
+--------------------------+-------------+-------------------+--------------------+--------------+----------------+
|                         0|            0|                  0|                   0|             0|               0|
+--------------------------+-------------+-------------------+--------------------+--------------+----------------+



In [96]:
# data1.filter(length("addr_state") == 1).show()   # Checking for string length > 2 and <2 is Zero
data1.select(  'term_in_months' ).distinct().show(29, False)

+--------------+
|term_in_months|
+--------------+
| 36           |
| 60           |
|NULL          |
+--------------+



In [97]:
data1 = data1.fillna(1,subset = ['policy_code'])

data1 = data1.fillna('N',subset = ['hardship_flag', 'debt_settlement_flag'])

data1 = data1.fillna('N/A', subset = ['emp_title','home_ownership','verification_status','grade','sub_grade','issue_d','loan_status', 'pymnt_plan','url','purpose','title' \
                                      , 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'initial_list_status','last_pymnt_d','next_pymnt_d', 'last_credit_pull_d',\
                                      'application_type', 'disbursement_method','term_in_months'])  # categorical Imputation



data1 = data1.fillna(0, subset = ['emp_length_years','annual_inc','loan_amnt','funded_amnt','int_rate','installment', 'funded_amnt_inv'\
                                  ,'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec','revol_bal',\
                                  'revol_util','total_acc', 'out_prncp','out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp','total_rec_int', 'total_rec_late_fee'\
                                  ,'recoveries', 'collection_recovery_fee','last_pymnt_amnt','last_fico_range_high', 'last_fico_range_low','collections_12_mths_ex_med', 'acc_now_delinq'\
                                  ,'tot_coll_amt', 'tot_cur_bal', 'open_act_il', 'open_acc_6m', 'mths_since_rcnt_il','total_bal_il' ,'open_il_24m','open_il_12m'\
                                  , 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'open_rv_12m', 'open_rv_24m',  'all_util', 'total_rev_hi_lim', 'total_cu_tl','inq_fi'\
                                  , 'avg_cur_bal', 'bc_open_to_buy', 'inq_last_12m', 'acc_open_past_24mths', 'bc_util', 'mo_sin_old_il_acct','delinq_amnt', 'chargeoff_within_12_mths'\
                                  ,'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',  'mths_since_recent_bc'\
                                  ,'mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl'\
                                  ,'num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_90g_dpd_24m'\
                                  ,'num_tl_120dpd_2m','num_tl_30dpd','num_tl_op_past_12m','pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies'\
                                  , 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'tax_liens', 'total_il_high_credit_limit',]) # Numerical Imputation

In [99]:
null_counts = data1.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in data1.columns])

# Display the result
null_counts.show()

+---+---------+-----------+---------------+--------+-----------+-----+---------+---------+--------------+----------+-------------------+-------+-----------+----------+---+-------+-----+--------+----------+---+-----------+----------------+--------------+---------------+--------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+-----------+----------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+-------+------------------------+-----------+-----------------